# Assignment lab 05
- 19.04.21

## Master Class: Machine Learning (5MI2018)
- Faculty of Economic Science
- University of Neuchatel (Switzerland)
- Lab 5, see ML21_Exercise_5.pdf for more information

## Authors: 
- Romain Claret @RomainClaret
- Sylvain Robert-Nicoud @Nic0uds

In [72]:
# get the features names and the values of the categories from adult.names (build a dictionary)

data_dict = {}
with open('adult.names') as f:
    for l in f:
        if l[0] == '|' or ':' not in l: continue
        c = l.split(':')
        if c[1].startswith(' continuous'): data_dict[c[0]] = ""
        else: data_dict[c[0]] = c[1].replace("\n","").replace(".","").replace(" ","").split(",")

In [73]:
# in the specifications (adult.names): Unknown values are replaced with the character '?'

import pandas as pd

header = list(data_dict.keys())+['income']
df_train = pd.read_table("adult.data", sep=r',\s', na_values='?', header=None, names=header).dropna()
df_evaluate = pd.read_table("adult.test", sep=r',\s', na_values='?', skiprows=[0], header=None, names=header).dropna()

/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py:767: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())
/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py:767: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


In [74]:
#import pandas_profiling
#df_train.profile_report()

In [75]:
#df_evaluate.profile_report()

In [77]:
# droping the education because it's redundant with education-num
# droping the occupation because it's not generic enough, we have much more categories that those captured in the training sample
# droping the relationship because it's not generic enough, we have much more categories that those captured in the training sample

drop_list = ["education", "occupation", "relationship"]
df_train = df_train.drop(columns=drop_list)
df_evaluate = df_evaluate.drop(columns=drop_list)

In [78]:
# reducing categories with multiple options into lower dimensions classification (into binary preferably) when possible
# - marital-status could be reduced as Married or Not-Married
# marital-status ['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent' 'Separated' 'Married-AF-spouse' 'Widowed']
# - workclass could be recuded to 3 dimensions: Government, Private, and Self-Employment
# Note that we take into consideration all the options for the category from the specifications
# ['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' 'Self-emp-inc' 'Without-pay']

dict_replace = {
    'marital-status' : {
        'Never-married': 'Not-Married',
        'Married-civ-spouse': 'Married',
        'Divorced': 'Not-Married',
        'Married-spouse-absent': 'Married',
        'Separated': 'Married',
        'Married-AF-spouse': 'Married',
        'Widowed': 'Not-Married'
        },
    'workclass': {
        'State-gov': 'Government',
        'Self-emp-not-inc': 'Self-Employment',
        'Federal-gov': 'Government',
        'Local-gov': 'Government',
        'Self-emp-inc': 'Self-Employment'
        }
}

df_train.replace(dict_replace, inplace=True)
df_evaluate.replace(dict_replace, inplace=True)

In [79]:
# uniformizing the categories between the training and evaluation datasets
# indeed, there is a . at the end of the value in the evaluation dataset for the income category and not in the training dataset
df_evaluate["income"].replace({"<=50K.": "<=50K", ">50K.": ">50K"}, inplace=True)

In [80]:
from sklearn.preprocessing import LabelEncoder
# for binary categories we will be using a label encoder
# - marital-status, sex, income

for l in ["marital-status", "sex", "income"]:
    l_enc = LabelEncoder()
    encoder_train = l_enc.fit(df_train[l])
    encoder_evaluate = l_enc.fit(df_evaluate[l])
    df_train["encoded_"+l] = encoder_train.transform(df_train[l])
    df_evaluate["encoded_"+l] = encoder_evaluate.transform(df_evaluate[l])
    
#df_train.reset_index(inplace=True,drop=True)
#df_evaluate.reset_index(inplace=True,drop=True)

In [81]:
# For non-binary categories, first we check the specifications of the dataset to validate all the options per category (we have data_dict)
# Indeed, the values in the categories are not always all present in a dataset
# race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
# native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
# and our custom category: workclass: Government, Private, and Self-Employment
# adding temporary fake data for the one hot encoder

fake_row = df_train[:1].copy()
df_fake = pd.DataFrame(data=fake_row, columns=df_train.columns)

cats_nonbinary = ["race", "native-country"]

for c in cats_nonbinary:
    for v in data_dict[c]:
        fake_row[c] = v
        df_fake = df_fake.append(fake_row, ignore_index=True)
        
cat_workclass = ["Government", "Private", "Self-Employment"]
for cw in cat_workclass:
    fake_row["workclass"] = cw
    df_fake = df_fake.append(fake_row, ignore_index=True)
    
df_train = df_train.append(df_fake).reset_index(drop=True)
df_evaluate = df_evaluate.append(df_fake).reset_index(drop=True)

In [83]:
# get meaningful columns
continuous_features = [k for k, v in data_dict.items() if v == ""]
unencoded_features = ["workclass", "race", "native-country"]
encoded_features = [c for c in df_train if c.startswith('encoded')]
columns = continuous_features+unencoded_features+encoded_features
#columns.remove("encoded_income")
#columns

In [85]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

# Standardizing of numeric values
# Doesn't have a lot of meaning in the case of decision trees as it's not using distances (like KNN)
# But it's just a pedagological flavor and to use the ColumnTranformer for whatever reason 
# https://stats.stackexchange.com/questions/10289/whats-the-difference-between-normalization-and-standardization

# for non-binary categories we will be using a onehot encoder as decision trees are sensitive to leaves values

# We choose a the best parameters from lab2 for the decision tree
# depth=8 Train accuracy_score 0.8550292179535
# depth=8 Test accuracy_score 0.8465108569534229
# depth=8 Evaluation accuracy_score 0.8469455511288181 

feature_transformation = ColumnTransformer(transformers=[
    ('categorical', OneHotEncoder(handle_unknown='ignore'), unencoded_features+encoded_features[:-1]),
    ('numerical', StandardScaler(), continuous_features)
])

adult_pipeline = Pipeline(steps=[
  ('features', feature_transformation),
  ('classifier', DecisionTreeClassifier(criterion='gini', random_state=1, max_depth=8))
])

In [149]:
#feature_transformation

ColumnTransformer(transformers=[('categorical',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['workclass', 'race', 'native-country',
                                  'encoded_marital-status', 'encoded_sex']),
                                ('numerical', StandardScaler(),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week'])])

In [ ]:
#classifiers={
#    'Dtree':DecisionTreeClassifier(random_state=42)
#}
#grids={
#    'Dtree':{
#        #'classifier__criterion':['gini', 'entropy'],
#        'classifier__max_depth': range(7, 8),
#        #'classifier__ccp_alpha': [x * 0.1 for x in range(0, 10)],
#        #'classifier__min_samples_split': [x * 0.1 for x in range(1, 10)],
#        #'classifier__min_samples_leaf': range(1, 8),
#        #'classifier__min_weight_fraction_leaf': [x * 0.1 for x in range(1, 5)],
#        #'classifier__min_impurity_decrease': range(1, 8)
#    }
#}
#for c in list(classifiers.keys()):
#    clf = classifiers[c]
#    grid = grids[c]
#    adult_pipeline.set_params(classifier=clf)
#    gridSearch_model = GridSearchCV(adult_pipeline, grid, )
#    gridSearch_model.fit(df_train, df_train["encoded_income"])
#    print(gridSearch_model.best_params_)

In [217]:
from sklearn.model_selection import GridSearchCV
#from sklearn.neighbors import KNeighborsClassifier
import time
import multiprocessing as mp

#set workers
nb_workers = mp.cpu_count()-1

#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
#https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
#https://towardsdatascience.com/how-to-tune-a-decision-tree-f03721801680
grids_params = [
    {
        'classifier': (DecisionTreeClassifier(random_state=1),),
        'classifier__criterion':['gini', 'entropy'], #best_solo='entropy'
        'classifier__max_depth': range(8, 12), #best_solo=8
        #'classifier__ccp_alpha': [x * 0.1 for x in range(0, 1)], #default: 0.0 #best_solo=0.0
        'classifier__min_samples_split': range(2,10), #default: 2 #best_solo=11
        #'classifier__min_samples_leaf': range(1, 3), #default: 1 #best_solo=34
        #'classifier__min_weight_fraction_leaf': [x * 0.1 for x in range(0, 2)], #default: 0.0 #best_solo=0.1
        #'classifier__min_impurity_decrease': range(0, 1), #default: 0 #best_solo=0
        'classifier__max_features': range(25,31) #default: n_features = 11 #best_solo=4
    }#,
    #{
    #    'classifier': (KNeighborsClassifier(),),
    #    'classifier__n_neighbors': range(25, 26),
    #}
]

start = time.time()
grid_search_model = GridSearchCV(estimator=adult_pipeline, param_grid=grids_params, n_jobs=25, cv=10, verbose=10)
grid_search_model.fit(df_train, df_train["encoded_income"])
end = time.time()
print("Time elapsed",str(end - start))
print(grid_search_model.best_params_)
#print(grid_search_model.cv_results_)
#print(grid_search_model.refit_time_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Time elapsed 69.29844117164612
{'classifier': DecisionTreeClassifier(min_samples_leaf=36, random_state=1), 'classifier__min_samples_leaf': 36}


In [173]:
pd.DataFrame(grid_search_model.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,8.588173,0.811497,0.152540,0.063191,"DecisionTreeClassifier(min_samples_leaf=34, ra...",20,{'classifier': DecisionTreeClassifier(min_samp...,0.830907,0.838187,0.834161,0.823568,0.832837,0.840450,0.838795,0.849057,0.843098,0.841774,0.837283,0.006812,20
1,9.070602,0.728922,0.199287,0.084219,"DecisionTreeClassifier(min_samples_leaf=34, ra...",21,{'classifier': DecisionTreeClassifier(min_samp...,0.831899,0.841496,0.839788,0.819927,0.836478,0.842436,0.840119,0.847071,0.842436,0.842105,0.838376,0.007233,19
2,9.953925,0.483752,0.218796,0.057092,"DecisionTreeClassifier(min_samples_leaf=34, ra...",22,{'classifier': DecisionTreeClassifier(min_samp...,0.831899,0.843150,0.839788,0.821251,0.838795,0.844091,0.838133,0.848726,0.843760,0.839126,0.838872,0.007260,18
3,7.861185,0.907171,0.161396,0.088079,"DecisionTreeClassifier(min_samples_leaf=34, ra...",23,{'classifier': DecisionTreeClassifier(min_samp...,0.833885,0.840503,0.842767,0.821251,0.841443,0.844422,0.839457,0.848064,0.845084,0.839788,0.839667,0.007131,17
4,7.103444,1.423095,0.157354,0.046475,"DecisionTreeClassifier(min_samples_leaf=34, ra...",24,{'classifier': DecisionTreeClassifier(min_samp...,0.838187,0.841165,0.846077,0.819927,0.840119,0.844422,0.840781,0.844422,0.846408,0.840450,0.840196,0.007247,16
5,8.321752,0.732788,0.210370,0.079732,"DecisionTreeClassifier(min_samples_leaf=34, ra...",25,{'classifier': DecisionTreeClassifier(min_samp...,0.836863,0.841827,0.843429,0.822906,0.837471,0.844422,0.838464,0.849057,0.848064,0.842436,0.840494,0.007043,15
6,9.138505,0.654658,0.143364,0.071879,"DecisionTreeClassifier(min_samples_leaf=34, ra...",26,{'classifier': DecisionTreeClassifier(min_samp...,0.836532,0.841827,0.845084,0.825554,0.839126,0.843760,0.838795,0.855677,0.849057,0.839126,0.841454,0.007574,11
7,8.342891,0.731975,0.190388,0.067392,"DecisionTreeClassifier(min_samples_leaf=34, ra...",27,{'classifier': DecisionTreeClassifier(min_samp...,0.839841,0.840172,0.846077,0.825554,0.840119,0.844091,0.839788,0.853691,0.847733,0.840450,0.841752,0.006923,7
8,8.927128,0.709583,0.170758,0.060087,"DecisionTreeClassifier(min_samples_leaf=34, ra...",28,{'classifier': DecisionTreeClassifier(min_samp...,0.837194,0.841165,0.845746,0.825223,0.840450,0.844091,0.837471,0.855677,0.844753,0.840781,0.841255,0.007329,12
9,9.131379,0.253832,0.181050,0.054463,"DecisionTreeClassifier(min_samples_leaf=34, ra...",29,{'classifier': DecisionTreeClassifier(min_samp...,0.836532,0.840172,0.844091,0.826216,0.840781,0.844753,0.838795,0.857001,0.847733,0.840781,0.841686,0.007513,8


In [174]:
#remove the fake rows
df_train = df_train[:-len(df_fake)]
df_evaluate = df_evaluate[:-len(df_fake)]

In [175]:
from sklearn.model_selection import train_test_split

# make training and testings sets
X_train, X_test, y_train, y_test = train_test_split(df_train,df_train["encoded_income"],test_size=0.2,random_state=1)

# make evaluation sets
X_evaluate = df_evaluate
y_evaluate = df_evaluate["encoded_income"]

In [176]:
#present depth with best score for evaluation dataset
from sklearn.metrics import accuracy_score

y_hat_dtree_train = grid_search_model.predict(X_train)
y_hat_dtree_test = grid_search_model.predict(X_test)
y_hat_dtree_evaluate = grid_search_model.predict(X_evaluate)

print("Train accuracy_score",accuracy_score(y_train,y_hat_dtree_train))
print("Test accuracy_score",accuracy_score(y_test,y_hat_dtree_test))
print("Evaluation accuracy_score",accuracy_score(y_evaluate,y_hat_dtree_evaluate),"\n")

Train accuracy_score 0.8610385842761822
Test accuracy_score 0.8604342781369136
Evaluation accuracy_score 0.8400398406374502 



In [ ]:
# lab 03 results:
# depth=8 Train accuracy_score 0.8539931203116582
# depth=8 Test accuracy_score 0.8518150174042765
# depth=8 Evaluation accuracy_score 0.8470783532536521 

In [177]:
import pickle
pickle.dump(grid_search_model, open("grid_search_model.pickle", "wb" ))

In [194]:
file_handler = open(filename, 'r').readlines()[1:]

In [210]:
filename = 'adult.test'
file_handler = open(filename, 'r').readlines()[1:]
prefix_file = "adult_2021_cw_"
week_number = 1
split_into = 10
line_count = 0
file_length = len(file_handler)

for i in range(0,file_length):
    if i % ((file_length)//split_into) == 0 and i+((file_length//split_into)//2) < file_length:
        open(str(prefix_file)+str(week_number) + ".csv", "w+").writelines(file_handler[i:i+(file_length//split_into)])
        week_number += 1

In [211]:
import pickle
pipeline_model = pickle.load( open("grid_search_model.pickle", "rb" ))

In [212]:
import warnings
warnings.filterwarnings("ignore")

weeks_count = 10
filename = 'adult.test'
prefix_file = "adult_2021_cw_"

# get the features names and the values of the categories from adult.names (build a dictionary)

data_dict = {}
with open('adult.names') as f:
    for l in f:
        if l[0] == '|' or ':' not in l: continue
        c = l.split(':')
        if c[1].startswith(' continuous'): data_dict[c[0]] = ""
        else: data_dict[c[0]] = c[1].replace("\n","").replace(".","").replace(" ","").split(",")
            
header = list(data_dict.keys())+['income']
            
for i in range (weeks_count):
    filename = str(prefix_file)+str(i+1)+".csv"
    df_weekly = pd.read_table(filename, sep=r',\s', na_values='?', skiprows=[0], header=None, names=header).dropna()
    
    drop_list = ["education", "occupation", "relationship"]
    df_weekly = df_weekly.drop(columns=drop_list)
    
    dict_replace = {
    'marital-status' : {
        'Never-married': 'Not-Married',
        'Married-civ-spouse': 'Married',
        'Divorced': 'Not-Married',
        'Married-spouse-absent': 'Married',
        'Separated': 'Married',
        'Married-AF-spouse': 'Married',
        'Widowed': 'Not-Married'
        },
    'workclass': {
        'State-gov': 'Government',
        'Self-emp-not-inc': 'Self-Employment',
        'Federal-gov': 'Government',
        'Local-gov': 'Government',
        'Self-emp-inc': 'Self-Employment'
        }
    }

    df_weekly.replace(dict_replace, inplace=True)
    
    df_weekly["income"].replace({"<=50K.": "<=50K", ">50K.": ">50K"}, inplace=True)
    
    for l in ["marital-status", "sex", "income"]:
        l_enc = LabelEncoder()
        encoder_weekly = l_enc.fit(df_weekly[l])
        df_weekly["encoded_"+l] = encoder_weekly.transform(df_weekly[l])
    
    y_hat_dtree_weekly = pipeline_model.predict(df_weekly)

    print(filename, "accuracy_score:",accuracy_score(df_weekly["encoded_income"],y_hat_dtree_weekly),"\n")
    
    pd.DataFrame(y_hat_dtree_weekly).to_csv(str(prefix_file)+str(i+1)+"_pred.csv",header=["pred_income"], index=None)

adult_2021_cw_1.csv accuracy_score: 0.8381079280479681 

adult_2021_cw_2.csv accuracy_score: 0.8430851063829787 

adult_2021_cw_3.csv accuracy_score: 0.8458471760797343 

adult_2021_cw_4.csv accuracy_score: 0.8304405874499332 

adult_2021_cw_5.csv accuracy_score: 0.8373015873015873 

adult_2021_cw_6.csv accuracy_score: 0.836241610738255 

adult_2021_cw_7.csv accuracy_score: 0.8545335942596216 

adult_2021_cw_8.csv accuracy_score: 0.83994708994709 

adult_2021_cw_9.csv accuracy_score: 0.8317757009345794 

adult_2021_cw_10.csv accuracy_score: 0.843687374749499 



16

In [31]:
# lab 03 results:
# adult_2021_cw_1.csv accuracy_score: 0.8293736501079914 
# adult_2021_cw_2.csv accuracy_score: 0.8503253796095445 
# adult_2021_cw_3.csv accuracy_score: 0.8427807486631016 
# adult_2021_cw_4.csv accuracy_score: 0.8307860262008734 
# adult_2021_cw_5.csv accuracy_score: 0.8507462686567164 
# adult_2021_cw_6.csv accuracy_score: 0.854978354978355 
# adult_2021_cw_7.csv accuracy_score: 0.8545454545454545 
# adult_2021_cw_8.csv accuracy_score: 0.8514531754574811 
# adult_2021_cw_9.csv accuracy_score: 0.8296943231441049 
# adult_2021_cw_10.csv accuracy_score: 0.8574537540805223 

In [3]:
%run "main_romain_claret_and_sylvain_robert-nicoud_lab4.py"

Fitting 10 folds for each of 54 candidates, totalling 540 fits

Time elapsed 255.32171940803528 

*******************************************************

Evaluate Decision Tree Classifier Pipeline on new data
******************************************************* 

Best hyperparameters {'classifier': DecisionTreeClassifier(max_depth=9, max_features=27, min_samples_split=5,
                       random_state=1), 'classifier__criterion': 'gini', 'classifier__max_depth': 9, 'classifier__max_features': 27, 'classifier__min_samples_split': 5}
Train accuracy_score 0.8541174520286792
Test accuracy_score 0.8508204873197415
Evaluation accuracy_score 0.846879150066401 



In [2]:
%run "predict_income_romain_claret_and_sylvain_robert-nicoud_lab4.py"

adult_2021_cw_1_pred.csv accuracy_score: 0.844103930712858 

adult_2021_cw_2_pred.csv accuracy_score: 0.851063829787234 

adult_2021_cw_3_pred.csv accuracy_score: 0.8498338870431894 

adult_2021_cw_4_pred.csv accuracy_score: 0.8384512683578104 

adult_2021_cw_5_pred.csv accuracy_score: 0.8373015873015873 

adult_2021_cw_6_pred.csv accuracy_score: 0.8429530201342282 

adult_2021_cw_7_pred.csv accuracy_score: 0.8551859099804305 

adult_2021_cw_8_pred.csv accuracy_score: 0.8531746031746031 

adult_2021_cw_9_pred.csv accuracy_score: 0.8411214953271028 

adult_2021_cw_10_pred.csv accuracy_score: 0.8557114228456913 

